Prepare query 

In [12]:
import requests
import json  
import os

class PowerfleetAPIManager:
    CRENDETIALS_FILE_PATH = './Powerfleet_API.txt'
    
    ############REQUEST FORMAT############
    # URL & headers (will populate the headers dynamically using api_key)
    URL     = 'https://powerfleet.net/POWERFLEET5000/tr_rest/secure/vehicle/vehicle-gps-data'
    HEADERS = {
        'Content-Type': 'application/json'  # Authorization header will be added dynamically
    }

    # Data (can dynamically update it as needed)
    DATA    = {
        "startDate": "2024-10-23 15:32:24",
        "endDate": "2024-11-25 15:32:24",
        "vehicleId": 7
    }
    ############REQUEST FORMAT############
    
    def __init__(self):
        pass

    @staticmethod
    def get_values_from_file(CRENDETIALS_FILE_PATH, *keys):
        # Create a dictionary to store key-value pairs from the file
        data = {}

        # Read the file and parse the key-value pairs
        try:
            with open(CRENDETIALS_FILE_PATH, 'r') as file:
                for line in file:
                    line = line.strip()  # Remove any leading/trailing whitespace
                    if ':' in line:
                        key, value = line.split(':', 1)  # Split into key and value
                        data[key.strip()] = value.strip()  # Store in dictionary
        except FileNotFoundError:
            print(f"Error: The file '{CRENDETIALS_FILE_PATH}' was not found.")
            return None

        # Fetch the values for the given keys
        result = {}
        for key in keys:
            if key in data:
                result[key] = data[key]
            else:
                result[key] = None  # If the key is not found, return None for that key

        return result

    #! Returns the response or None 
    def create_query(self):
        # Get the necessary values from the file
        keys_to_lookup = ['cid', 'api_key']  # keys that we need to look up in the file
        values = self.get_values_from_file(self.CRENDETIALS_FILE_PATH, *keys_to_lookup)
        
        if values and values['cid'] and values['api_key']:
            self.HEADERS['Authorization'] = values['api_key']
            
            self.DATA['cid'] = values['cid']

            # Print the headers and data for debugging purposes
            print(f"Headers: {self.HEADERS}")
            print(f"Data: {self.DATA}")

            # Make the request (POST)
            response = requests.post(self.URL, headers=self.HEADERS, json=self.DATA)

            # Handle the response
            if response.status_code == 200:
                print("Success:")
                # Pretty-print the JSON response
                pretty_json = json.dumps(response.json(), indent=4)
                print(pretty_json)
                return pretty_json
            else:
                print(f"Failed with status code {response.status_code}:")
                print(response.text)
                return None
        else:
            print("Error: Missing required data (cid or api_key).")
            return None
    
    


Headers: {'Content-Type': 'application/json', 'Authorization': '00595c0d-01d4-38a3-ba06-8852a0c6b2b8'}
Data: {'startDate': '2024-10-23 15:32:24', 'endDate': '2024-11-25 15:32:24', 'vehicleId': 7, 'cid': '946'}
Success:
[
    {
        "vehicleId": 7,
        "lat": 37.508455,
        "lng": 22.3740133,
        "dateStored": 1729707515000,
        "velocity": 0.0,
        "odometer": 0.0,
        "engineVoltage": 4.762
    },
    {
        "vehicleId": 7,
        "lat": 37.50836,
        "lng": 22.3737299,
        "dateStored": 1729707516000,
        "velocity": 19.0,
        "odometer": 0.0,
        "engineVoltage": 4.762
    },
    {
        "vehicleId": 7,
        "lat": 37.508375,
        "lng": 22.3736733,
        "dateStored": 1729707528000,
        "velocity": 7.0,
        "odometer": 0.0,
        "engineVoltage": 4.756
    },
    {
        "vehicleId": 7,
        "lat": 37.5083816,
        "lng": 22.3735899,
        "dateStored": 1729707533000,
        "velocity": 29.0,
        

Get query and send it to DB

In [14]:
from pymongo import MongoClient, errors
from bson import ObjectId


class DataBaseConnector:
    DB_NAME         = "Ptyxiaki"
    COLLECTION_NAME = "Powerfleet GPS"
    PIPELINE        = []
    QUERY_TO_DB     = ""
    

    def __init__(self, created_query):
        self.QUERY_TO_DB = created_query.create_query()
    
    def get_DB_info(self):
        return self.DB_NAME, self.COLLECTION_NAME 

    def connect_to_db(self):
        try:
            client = MongoClient("localhost:27017", serverSelectionTimeoutMS=5000) 
            db = client[self.DB_NAME]
            collection = db[self.COLLECTION_NAME]
            
            client.server_info() 

            return collection
        
        except errors.ServerSelectionTimeoutError as e:
            print(f"Error: Could not connect to the MongoDB server. Details: {e}")
        except errors.PyMongoError as e:
            print(f"Error: An error occurred while connecting to the database. Details: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")
        #* uncaught Exception, return None
        return None 
        
    def insert_document(self):
        if not hasattr(self, 'collection'): #! Check if a connection with DB is established first
            print("Error: Database not connected. Please connect to the database first.")
            return
        
        try:
            result = self.collection.insert_one(self.QUERY_TO_DB)
            print(f"Results after inserting the response: {result.inserted_id}")
        except errors.PyMongoError as e:
            print(f"Error: Failed to insert document. Details: {e}")
        except Exception as e:
            print(f"Unexpected error while inserting document: {e}")

Headers: {'Content-Type': 'application/json', 'Authorization': '00595c0d-01d4-38a3-ba06-8852a0c6b2b8'}
Data: {'startDate': '2024-10-23 15:32:24', 'endDate': '2024-11-25 15:32:24', 'vehicleId': 7, 'cid': '946'}
Success:
[
    {
        "vehicleId": 7,
        "lat": 37.508455,
        "lng": 22.3740133,
        "dateStored": 1729707515000,
        "velocity": 0.0,
        "odometer": 0.0,
        "engineVoltage": 4.762
    },
    {
        "vehicleId": 7,
        "lat": 37.50836,
        "lng": 22.3737299,
        "dateStored": 1729707516000,
        "velocity": 19.0,
        "odometer": 0.0,
        "engineVoltage": 4.762
    },
    {
        "vehicleId": 7,
        "lat": 37.508375,
        "lng": 22.3736733,
        "dateStored": 1729707528000,
        "velocity": 7.0,
        "odometer": 0.0,
        "engineVoltage": 4.756
    },
    {
        "vehicleId": 7,
        "lat": 37.5083816,
        "lng": 22.3735899,
        "dateStored": 1729707533000,
        "velocity": 29.0,
        

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000), 'Ptyxiaki'), 'Powerfleet GPS')

In [17]:
if __name__ == "__main__":
    pam = PowerfleetAPIManager()
    print(pam.create_query())
    obje = DataBaseConnector(pam)
    obje.connect_to_db()   
    obje.insert_document() 


Headers: {'Content-Type': 'application/json', 'Authorization': '00595c0d-01d4-38a3-ba06-8852a0c6b2b8'}
Data: {'startDate': '2024-10-23 15:32:24', 'endDate': '2024-11-25 15:32:24', 'vehicleId': 7, 'cid': '946'}
Success:
[
    {
        "vehicleId": 7,
        "lat": 37.508455,
        "lng": 22.3740133,
        "dateStored": 1729707515000,
        "velocity": 0.0,
        "odometer": 0.0,
        "engineVoltage": 4.762
    },
    {
        "vehicleId": 7,
        "lat": 37.50836,
        "lng": 22.3737299,
        "dateStored": 1729707516000,
        "velocity": 19.0,
        "odometer": 0.0,
        "engineVoltage": 4.762
    },
    {
        "vehicleId": 7,
        "lat": 37.508375,
        "lng": 22.3736733,
        "dateStored": 1729707528000,
        "velocity": 7.0,
        "odometer": 0.0,
        "engineVoltage": 4.756
    },
    {
        "vehicleId": 7,
        "lat": 37.5083816,
        "lng": 22.3735899,
        "dateStored": 1729707533000,
        "velocity": 29.0,
        